In [9]:
# Imports
import os

import numpy as np

import rosbag2_py
from rclpy.serialization import deserialize_message
from rosidl_runtime_py.utilities import get_message

import rclpy
from rclpy.node import Node

from sensor_msgs.msg import Image
from audio_common_msgs.msg import AudioDataStamped

from situated_hri_interfaces.srv import SceneVisRec, SceneAudioRec, RecordEpoch
from std_srvs.srv import Empty

# Setup
```
cd ~/sit_int_ws
source /opt/ros/humble/setup.bash
source install/setup.bash
ros2 launch situated_interaction exp1a_record_scene_data_cfg1.launch.py
```

# Play scene data, record results for evaluation

In [10]:
# Parameters
filepath = '../../bags/e1_est_tuning'

In [11]:
# Member variables
rclpy.init()
node = Node('analysis_script_node')

# Set parameters


# Create custom Quality of Service profile to avoid errors with tf_static playback
tf_static_qos = rclpy.qos.QoSProfile(history = rclpy.qos.HistoryPolicy.KEEP_LAST, 
                                     depth = 10,
                                     reliability = rclpy.qos.ReliabilityPolicy.RELIABLE,
                                     durability = rclpy.qos.DurabilityPolicy.TRANSIENT_LOCAL)

# Publishers
audio_data_pub = node.create_publisher(AudioDataStamped, 'audio_data',10)
img_data_pub = node.create_publisher(Image, 'oak/rgb/image_raw',10)

# Reset CLIP msg count (downsampling)
reset_clip_scene_client = node.create_client(Empty, 'clip_scene_rec/reset')

# Reset scene estimates from different modalities
reset_bayes_audio_scene_client = node.create_client(Empty, 'bayes_audio_scene_est/reset') 
reset_bayes_clip_scene_client = node.create_client(Empty, 'bayes_clip_scene_est/reset')
reset_bayes_fused_scene_client = node.create_client(Empty, 'bayes_fused_scene_est/reset')

# Start/stop recording services
start_scene_recording_client = node.create_client(RecordEpoch, 'record_scene_results_node/record_epoch')
stop_scene_recording_client = node.create_client(Empty, 'record_scene_results_node/stop_recording')

# Other member variables
actual_scene_labels = ['campus','courtyard','lab','lobby']

# Helper functions
def typename(topic_name):
    for topic_type in topic_types:
        if topic_type.name == topic_name:
            return topic_type.type
    raise ValueError(f"topic {topic_name} not in bag")

In [12]:
# Traverse filepath
for root, dirs, files in os.walk(filepath):

    # Examine each file
    for file in files:
        
        # If it is an mcap, examine
        if os.path.splitext(file)[-1] == '.mcap':

            # Get labels based on filepath
            labels = os.path.normpath(root).split(os.sep)
            command_actual = labels[-1]
            cmd_mode_actual = labels[-2]
            role_actual = labels[-3]
            scene_actual = labels[-4]
            iteration = os.path.splitext(file)[-2].split('_')[-1]

            scene_actual_idx = actual_scene_labels.index(scene_actual)
            
            print('Command: %s\nModality: %s\nRole: %s\nScene: %s (%s)\n\n' % (command_actual, cmd_mode_actual, role_actual, scene_actual, scene_actual_idx))

            # Start recording results for this epoch
            record_epoch_req = RecordEpoch.Request()
            record_epoch_req.scene = scene_actual
            record_epoch_req.role = role_actual
            record_epoch_req.cmd_mode = cmd_mode_actual
            record_epoch_req.cmd = command_actual
            record_epoch_req.iteration = int(iteration)

            while not start_scene_recording_client.wait_for_service(timeout_sec=1.0):
                print('Start recording service not available, waiting again...')
            
            start_scene_recording_future = start_scene_recording_client.call_async(record_epoch_req)
            rclpy.spin_until_future_complete(node, start_scene_recording_future)
            
            reader = rosbag2_py.SequentialReader()            
            reader.open(
                rosbag2_py.StorageOptions(uri=os.path.join(root,file), storage_id="mcap"),
                rosbag2_py.ConverterOptions(
                    input_serialization_format="cdr", output_serialization_format="cdr"
                ),
            )
            topic_types = reader.get_all_topics_and_types()

            # Iterate through messages
            while reader.has_next():
                topic, data, timestamp = reader.read_next()
                msg_type = get_message(typename(topic))

                # Handle audio scene data
                if topic=='/audio_data':
                    audio_data_pub.publish(deserialize_message(data,msg_type))

                if topic=='/oak/rgb/image_raw':
                    img_data_pub.publish(deserialize_message(data,msg_type))
                
            del reader

            # Reset estimator nodes           
            bayes_audio_future = reset_bayes_audio_scene_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(node, bayes_audio_future,timeout_sec=10)

            bayes_clip_future = reset_bayes_clip_scene_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(node, bayes_clip_future,timeout_sec=10)

            bayes_fused_future = reset_bayes_fused_scene_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(node, bayes_fused_future,timeout_sec=10)
            
            clip_future = reset_clip_scene_client.call_async(Empty.Request())
            rclpy.spin_until_future_complete(node, clip_future,timeout_sec=10)


# Cleanup
stop_scene_recording_future = stop_scene_recording_client.call_async(Empty.Request())
rclpy.spin_until_future_complete(node, stop_scene_recording_future)

rclpy.shutdown()

Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-in-reverse
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: move-forward
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: follow-me
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: follow-me
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Command: follow-me
Modality: gest_verb
Role: supervisor
Scene: lab (2)


Comman

[WARN] [1724023720.983254271] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1724023721.117132756] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


Command: move-in-reverse
Modality: verbal
Role: supervisor
Scene: campus (0)


Command: move-in-reverse
Modality: verbal
Role: supervisor
Scene: campus (0)




[WARN] [1724023721.255413564] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1724023721.399459734] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


Command: move-forward
Modality: verbal
Role: supervisor
Scene: campus (0)




...

Command: rally
Modality: verbal
Role: supervisor
Scene: lobby (3)




[WARN] [1724024414.615692100] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order
[WARN] [1724024414.809827248] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


Command: rally
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: rally
Modality: verbal
Role: supervisor
Scene: lobby (3)




[WARN] [1724024414.998869551] [rosbag2_storage_mcap]: no message indices found, falling back to reading in file order


Command: halt
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: halt
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: halt
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: halt
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: halt
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: advance
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: advance
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: advance
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: advance
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: advance
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: attention
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: attention
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: attention
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: attention
Modality: verbal
Role: supervisor
Scene: lobby (3)


Command: attention
Modality

In [ ]:
# For file in filepath, get scene, role, modality, and command

# Reset estimator nodes

# LAST - Load new parameters

# Play bag file

# Record scene responses - audio, visual, fused

# Record command responses - ARtag, speech, gesture, fused